## LLAMA 70B

1. Choose the following kernel: PyTorch-2.0.1
2. Install accelerate and trnasformers by running the below cell
3. You can find more information about LLAMA 70B: https://huggingface.co/upstage/SOLAR-0-70b-16bit

**Please do not edit any code**

In [1]:
!pip install accelerate
!pip3 install git+https://github.com/huggingface/transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /scratch/local/13718414/pip-req-build-ylqghv_a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /scratch/local/13718414/pip-req-build-ylqghv_a
  Resolved https://github.com/huggingface/transformers to commit 6f316016877197014193b9463b2fd39fa8f0c8e4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import transformers
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoConfig,AutoModelForCausalLM

In [3]:
output_dir="/blue/cap4641/LLM/llama"
model_config="/blue/cap4641/LLM/llama/models--upstage--SOLAR-0-70b-16bit/snapshots/cfd79568e72039d4f857a1f04d16232ad5da51f1"

In [4]:
def initialize_llm(output_dir,model_config):
    config = AutoConfig.from_pretrained(model_config, trust_remote_code=True,cache_dir=output_dir)
    tokenizer = AutoTokenizer.from_pretrained(model_config,config = AutoConfig.from_pretrained(model_config, trust_remote_code=True,cache_dir=output_dir)
,cache_dir=output_dir)


    pipeline = transformers.pipeline(
        "text-generation",
        model=model_config,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    device_map="auto",
        config=config,model_kwargs={'cache_dir':output_dir},
    )
    return pipeline,tokenizer

def get_response(text,pipeline,tokenizer):
    '''
    Gets Response from LLAMA 70B
    Keyword Arguments:
    :pipeline:Created pipeline object
    :tokenizer:LLAMA70B Tokenizer
    '''

    input_text="### User:\n"+text+"\n\n### Assistant:\n"

    sequences = pipeline(
    input_text,
    max_length=1000,
    do_sample=True,
    return_full_text=False,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

    return sequences[0]['generated_text']

def chat(pipeline,tokenizer):
    flag=True
    while flag:
        x=input("Enter your Question:")
        if x.lower()=='end':
            flag=False
        else:
            print("LLAMA 2 :")
            print(get_response(x,pipeline,tokenizer))


In [ ]:
if __name__ == "__main__":
  pipeline,tokenizer=initialize_llm(output_dir,model_config)


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
if __name__ == "__main__":
  chat(pipeline,tokenizer)